In [7]:
"""
match_iksan_okc.py
  • parsed_parts.csv   : 파서 결과 (IK / OK)
  • Cross_Map.csv      : 품명군 매핑 규칙
출력
  • matched_parts.csv  : 익산 ↔ 옥천 품번 매칭 결과
"""

import pandas as pd
from pathlib import Path

In [8]:
# ── 파일 경로 ────────────────────────────────────────────────
BASE_DIR = Path.cwd().parent

PP_CSV   = BASE_DIR / "data" / 'parsed_parts.csv'
XMAP_CSV = BASE_DIR / "data" / 'Cross_Map.csv'
OUT_CSV  = BASE_DIR / "data" / 'matched_parts.csv'

In [9]:
# ── 1. 데이터 적재 ───────────────────────────────────────────
parts = pd.read_csv(PP_CSV, dtype=str, encoding = 'cp949').fillna('')
xmap  = pd.read_csv(XMAP_CSV, dtype=str).fillna('')

# 익산 / 옥천 분리
ik = parts[parts.system == 'IK'].copy()
ok = parts[parts.system == 'OK'].copy()

In [10]:
def attach_km(df_ik, xmap_df):
    rows = []
    for _, r in df_ik.iterrows():
        cond = (
            (xmap_df.ik_part_type == r.part_type) &
            ((xmap_df.ik_grade_code == '') | (xmap_df.ik_grade_code == r.grade_code)) &
            ((xmap_df.ik_seal_code  == '') | (xmap_df.ik_seal_code  == r.seal_snap_code))
        )
        hit = xmap_df[cond]
        km  = hit.ok_km_code.iloc[0] if not hit.empty else ''
        note= hit.note.iloc[0]       if not hit.empty else ''
        rows.append({**r, 'ok_km_code': km, 'map_note': note})
    return pd.DataFrame(rows)

ik_map = attach_km(ik, xmap)

In [11]:
# ── 2) 옥천 품명군과 단순 Join (part_type = ok_km_code)
merged = ik_map.merge(
    ok[['part_type','remark']],          # 옥천에 있는 추가 정보 원하면 더 붙일 수 있음
    left_on='ok_km_code', right_on='part_type',
    how='left', suffixes=('_IK','_OK')
)

merged['match_flag'] = merged.part_type_OK.where(
    merged.part_type_OK.notna(), 'NO_MATCH')

merged.to_csv(OUT_CSV, index=False, encoding='cp949')
print(f"✅ 매칭 완료 → {OUT_CSV}  (총 {len(merged)}행)")

✅ 매칭 완료 → C:\Users\Allen\Desktop\Project\TYM\V_CODE\data\matched_parts.csv  (총 6행)
